# Apache Spark Analytical Window Functions

References: 

https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html

https://alvinhenrick.com/2017/05/16/apache-spark-analytical-window-functions/
        

## Salary within each department

* ### Ranking functions
    * rank
    * dense_rank
    * row_number
    * percent_rank?
    * ntile
* ### Analytic functions
    * lead
    * lag
    * cume_dist(cumulative distribution)?
* ### Aggregate functions
    * sum
    * running sum
    * avg
    * max
    * min

In [1]:
# Connect to Spark by creating a Spark session
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("WindowFunctions")\
    .getOrCreate()

In [2]:
from pyspark.sql.window import Window
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

#Create Sample Dataframe
empDF = [
      (7369, "SMITH", "CLERK", 7902, "17-Dec-80", 800, 20, 10),
      (7499, "ALLEN", "SALESMAN", 7698, "20-Feb-81", 1600, 300, 30),
      (7521, "WARD", "SALESMAN", 7698, "22-Feb-81", 1250, 500, 30),
      (7522, "KIM", "SALESMAN", 7698, "25-Feb-81", 1200, 500, 30),
      (7566, "JONES", "MANAGER", 7839, "2-Apr-81", 2975, 0, 20),
      (7654, "MARTIN", "SALESMAN", 7698, "28-Sep-81", 1250, 1400, 30),
      (7698, "BLAKE", "MANAGER", 7839, "1-May-81", 2850, 0, 30),
      (7782, "CLARK", "MANAGER", 7839, "9-Jun-81", 2450, 0, 10),
      (7788, "SCOTT", "ANALYST", 7566, "19-Apr-87", 3000, 0, 20),
      (7839, "KING", "PRESIDENT", 0, "17-Nov-81", 5000, 0, 10),
      (7844, "TURNER", "SALESMAN", 7698, "8-Sep-81", 1500, 0, 30),
      (7876, "ADAMS", "CLERK", 7788, "23-May-87", 1100, 0, 20)]

empDF = spark.createDataFrame(empDF, ["empno", "ename", "job", "mgr", "hiredate", "sal", "comm", "deptno"])
    
empDF.show()   

empDF.registerTempTable("emp")

+-----+------+---------+----+---------+----+----+------+
|empno| ename|      job| mgr| hiredate| sal|comm|deptno|
+-----+------+---------+----+---------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-Dec-80| 800|  20|    10|
| 7499| ALLEN| SALESMAN|7698|20-Feb-81|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-Feb-81|1250| 500|    30|
| 7522|   KIM| SALESMAN|7698|25-Feb-81|1200| 500|    30|
| 7566| JONES|  MANAGER|7839| 2-Apr-81|2975|   0|    20|
| 7654|MARTIN| SALESMAN|7698|28-Sep-81|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839| 1-May-81|2850|   0|    30|
| 7782| CLARK|  MANAGER|7839| 9-Jun-81|2450|   0|    10|
| 7788| SCOTT|  ANALYST|7566|19-Apr-87|3000|   0|    20|
| 7839|  KING|PRESIDENT|   0|17-Nov-81|5000|   0|    10|
| 7844|TURNER| SALESMAN|7698| 8-Sep-81|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-May-87|1100|   0|    20|
+-----+------+---------+----+---------+----+----+------+



## Ranking functions

#### SQL

In [3]:
ranking = spark.sql("""
SELECT 
    empno,
    deptno,
    sal,
    RANK() OVER (partition by deptno ORDER BY sal desc) as rank, 
    DENSE_RANK() OVER (partition by deptno ORDER BY sal desc) as dense_rank,
    ROW_NUMBER() OVER (partition by deptno ORDER BY sal desc) as row_number,
    PERCENT_RANK() OVER (partition by deptno ORDER BY sal desc) as percent_rank,
    NTILE(3) OVER (partition by deptno ORDER BY sal desc) as ntile,
    CUME_DIST() OVER (PARTITION BY deptno ORDER BY sal desc) as cume_dist
    FROM emp
""")

ranking.show()

+-----+------+----+----+----------+----------+------------+-----+-------------------+
|empno|deptno| sal|rank|dense_rank|row_number|percent_rank|ntile|          cume_dist|
+-----+------+----+----+----------+----------+------------+-----+-------------------+
| 7839|    10|5000|   1|         1|         1|         0.0|    1| 0.3333333333333333|
| 7782|    10|2450|   2|         2|         2|         0.5|    2| 0.6666666666666666|
| 7369|    10| 800|   3|         3|         3|         1.0|    3|                1.0|
| 7698|    30|2850|   1|         1|         1|         0.0|    1|0.16666666666666666|
| 7499|    30|1600|   2|         2|         2|         0.2|    1| 0.3333333333333333|
| 7844|    30|1500|   3|         3|         3|         0.4|    2|                0.5|
| 7521|    30|1250|   4|         4|         4|         0.6|    2| 0.8333333333333334|
| 7654|    30|1250|   4|         4|         5|         0.6|    3| 0.8333333333333334|
| 7522|    30|1200|   6|         5|         6|        

##### DF API

In [4]:
windowSpec = Window.partitionBy(empDF['deptno']).orderBy(empDF['sal'].desc())
rank = func.rank().over(windowSpec)
denseRank = func.dense_rank().over(windowSpec)
rowNum = func.row_number().over(windowSpec)
runningSum = func.sum(empDF['sal']).over(windowSpec)
lead = func.lead(empDF['sal']).over(windowSpec)
lag = func.lag(empDF['sal']).over(windowSpec)
empDF.select('empno','deptno','sal',
             rank.alias('rank'),
             denseRank.alias('dense_rank'),
             rowNum.alias('row_num'),
             runningSum.alias('running_sum'),
             lead.alias('next_val'),
             lag.alias('prev_val')).show()

+-----+------+----+----+----------+-------+-----------+--------+--------+
|empno|deptno| sal|rank|dense_rank|row_num|running_sum|next_val|prev_val|
+-----+------+----+----+----------+-------+-----------+--------+--------+
| 7839|    10|5000|   1|         1|      1|       5000|    2450|    null|
| 7782|    10|2450|   2|         2|      2|       7450|     800|    5000|
| 7369|    10| 800|   3|         3|      3|       8250|    null|    2450|
| 7698|    30|2850|   1|         1|      1|       2850|    1600|    null|
| 7499|    30|1600|   2|         2|      2|       4450|    1500|    2850|
| 7844|    30|1500|   3|         3|      3|       5950|    1250|    1600|
| 7521|    30|1250|   4|         4|      4|       8450|    1250|    1500|
| 7654|    30|1250|   4|         4|      5|       8450|    1200|    1250|
| 7522|    30|1200|   6|         5|      6|       9650|    null|    1250|
| 7788|    20|3000|   1|         1|      1|       3000|    2975|    null|
| 7566|    20|2975|   2|         2|   

## Analytic functions

#### SQL

In [5]:
rankTest = spark.sql("""
SELECT 
    empno,
    deptno,
    sal,
    lead(sal) OVER (PARTITION BY deptno ORDER BY sal desc) as next_val,
    lead(sal,2) OVER (PARTITION BY deptno ORDER BY sal desc) as next_to_next_val,
    lag(sal) OVER (PARTITION BY deptno ORDER BY sal desc) as prev_val,
    first(sal) OVER (PARTITION BY deptno ORDER BY sal desc) as first,
    last(sal) OVER (PARTITION BY deptno ORDER BY sal desc ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) as last
    FROM emp
""")

rankTest.show()

+-----+------+----+--------+----------------+--------+-----+----+
|empno|deptno| sal|next_val|next_to_next_val|prev_val|first|last|
+-----+------+----+--------+----------------+--------+-----+----+
| 7839|    10|5000|    2450|             800|    null| 5000| 800|
| 7782|    10|2450|     800|            null|    5000| 5000| 800|
| 7369|    10| 800|    null|            null|    2450| 5000| 800|
| 7698|    30|2850|    1600|            1500|    null| 2850|1200|
| 7499|    30|1600|    1500|            1250|    2850| 2850|1200|
| 7844|    30|1500|    1250|            1250|    1600| 2850|1200|
| 7521|    30|1250|    1250|            1200|    1500| 2850|1200|
| 7654|    30|1250|    1200|            null|    1250| 2850|1200|
| 7522|    30|1200|    null|            null|    1250| 2850|1200|
| 7788|    20|3000|    2975|            1100|    null| 3000|1100|
| 7566|    20|2975|    1100|            null|    3000| 3000|1100|
| 7876|    20|1100|    null|            null|    2975| 3000|1100|
+-----+---

In [6]:
rankTest = spark.sql("""
SELECT 
    empno,
    deptno,
    sal,
    sum(sal) OVER (PARTITION BY deptno ORDER BY sal desc) as running_sum,
    sum(sal) OVER (PARTITION BY deptno) as sum,
    avg(sal) OVER (PARTITION BY deptno) as avg,
    max(sal) OVER (PARTITION BY deptno) as max,
    min(sal) OVER (PARTITION BY deptno) as min,
    count(*) OVER (PARTITION BY deptno) as count
    FROM emp
""")

rankTest.show()

+-----+------+----+-----------+----+------------------+----+----+-----+
|empno|deptno| sal|running_sum| sum|               avg| max| min|count|
+-----+------+----+-----------+----+------------------+----+----+-----+
| 7839|    10|5000|       5000|8250|            2750.0|5000| 800|    3|
| 7782|    10|2450|       7450|8250|            2750.0|5000| 800|    3|
| 7369|    10| 800|       8250|8250|            2750.0|5000| 800|    3|
| 7698|    30|2850|       2850|9650|1608.3333333333333|2850|1200|    6|
| 7499|    30|1600|       4450|9650|1608.3333333333333|2850|1200|    6|
| 7844|    30|1500|       5950|9650|1608.3333333333333|2850|1200|    6|
| 7521|    30|1250|       8450|9650|1608.3333333333333|2850|1200|    6|
| 7654|    30|1250|       8450|9650|1608.3333333333333|2850|1200|    6|
| 7522|    30|1200|       9650|9650|1608.3333333333333|2850|1200|    6|
| 7788|    20|3000|       3000|7075|2358.3333333333335|3000|1100|    3|
| 7566|    20|2975|       5975|7075|2358.3333333333335|3000|1100

In [7]:
data = \
  [("Thin", "Cell Phone", 6000),
  ("Normal", "Tablet", 1500),
  ("Mini", "Tablet", 5500),
  ("Ultra thin", "Cell Phone", 5500),
  ("Very thin", "Cell Phone", 6000),
  ("Big", "Tablet", 2500),
  ("Bendable", "Cell Phone", 3000),
  ("Foldable", "Cell Phone", 3000),
  ("Pro", "Tablet", 4500),
  ("Pro2", "Tablet", 6500)]
df = spark.createDataFrame(data, ["product", "category", "revenue"])
df.registerTempTable("productRevenue")

### Question: What is the difference between the revenue of a product and the revenue of the best selling product in the same category as this product?

In [8]:
df = spark.sql("""
SELECT
  product,
  category,
  revenue
FROM (
  SELECT
    product,
    category,
    revenue,
    dense_rank() OVER (PARTITION BY category ORDER BY revenue DESC) as rank
  FROM productRevenue) tmp
WHERE
  rank <= 2

""")

#Check the Query Plan
#df.explain(extended = True)

df.show()

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Pro2|    Tablet|   6500|
|      Mini|    Tablet|   5500|
|      Thin|Cell Phone|   6000|
| Very thin|Cell Phone|   6000|
|Ultra thin|Cell Phone|   5500|
+----------+----------+-------+



### Question: Calculate the MAX reveue per catagory.

In [9]:
df = spark.sql("""
  SELECT
    product,
    category,
    revenue,
    max(revenue) OVER (PARTITION BY category) as max
  FROM productRevenue
""")

df.show()

+----------+----------+-------+----+
|   product|  category|revenue| max|
+----------+----------+-------+----+
|    Normal|    Tablet|   1500|6500|
|      Mini|    Tablet|   5500|6500|
|       Big|    Tablet|   2500|6500|
|       Pro|    Tablet|   4500|6500|
|      Pro2|    Tablet|   6500|6500|
|      Thin|Cell Phone|   6000|6000|
|Ultra thin|Cell Phone|   5500|6000|
| Very thin|Cell Phone|   6000|6000|
|  Bendable|Cell Phone|   3000|6000|
|  Foldable|Cell Phone|   3000|6000|
+----------+----------+-------+----+



### Question: What are the best selling and second best selling products in every category?
Below is the SQL query used to answer this question by using window function dense_rank

In [10]:
# Window function partioned by Category and ordered by Revenue
windowSpec = \
  Window \
    .partitionBy(df['category']) \
    .orderBy(df['revenue'].desc()) \
    .rangeBetween(-sys.maxsize, sys.maxsize)
    
# Create dataframe based on the productRevenue table    
dataFrame = spark.table("productRevenue")

# Calculate the Revenue difference
revenue_difference = \
  (func.max(dataFrame['revenue']).over(windowSpec) - dataFrame['revenue'])
  
#Generate a new dataframe (original dataframe and the revenue difference)
revenue_diff = dataFrame.select(
  dataFrame['product'],
  dataFrame['category'],
  dataFrame['revenue'],
  revenue_difference.alias("revenue_difference"))

revenue_diff.show()

+----------+----------+-------+------------------+
|   product|  category|revenue|revenue_difference|
+----------+----------+-------+------------------+
|      Pro2|    Tablet|   6500|                 0|
|      Mini|    Tablet|   5500|              1000|
|       Pro|    Tablet|   4500|              2000|
|       Big|    Tablet|   2500|              4000|
|    Normal|    Tablet|   1500|              5000|
|      Thin|Cell Phone|   6000|                 0|
| Very thin|Cell Phone|   6000|                 0|
|Ultra thin|Cell Phone|   5500|               500|
|  Bendable|Cell Phone|   3000|              3000|
|  Foldable|Cell Phone|   3000|              3000|
+----------+----------+-------+------------------+

